In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
#print(data[:100])

In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[ 1, 73, 58, 54, 71, 10, 57, 71],
        [73,  1, 57, 58, 59, 58, 54, 73],
        [67, 60, 11,  1, 32, 58,  1, 71],
        [58, 71,  9,  1, 54, 67, 57,  1]])
targets:
tensor([[73, 58, 54, 71, 10, 57, 71, 68],
        [ 1, 57, 58, 59, 58, 54, 73, 58],
        [60, 11,  1, 32, 58,  1, 71, 58],
        [71,  9,  1, 54, 67, 57,  1, 54]])


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


6m4?IIhy
Sxhh&Z;"0b1PviwAO.Rh:SoHWZ"9w;IwA_b:F;TwAXkXJ8LE *5yplG58sP6iciw]u0bzcgLoFm"-; dsgr[Zsiw,(1a41jyz&G!_&5Zc5U)SEN)*De:NB"Ylvf1]!aJm,KbLoAPEuqB0stOr[AKOG7o3VCRa1g*5!IZHzQ2:?33,PuCL(p'S-[Zg:OIa.Mlzc ,oB'SGq)2&vMlIm,(FmC0:W Y1PXf[WA(?_12!aWw_:bX?4
B0d(0BEZ1l_E96XL.L
iD'z0WTHWqn,yp!St&QWX*aeJdxmIII(12?*,1G[CC_-vxQnaKSK?_wEp2iyBE8T[A'aSaJNLrjvF2Mv!:&02UIvX*F5USp2klY2nqxm690JL"gH9PtQnGF.:S
X]h3lY2a.V
xdg_q_K(u?fXbYm4_]yH'988C0WnhUwzM3:F_S&f*k[,(y6d(Gx7e2X*:?mu[wNDrjZ;(y
USo5HEv4.,Z2U*y
E6!nk2rY


In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.637, val loss: 4.638
step: 250, train loss: 4.587, val loss: 4.599
step: 500, train loss: 4.543, val loss: 4.530
step: 750, train loss: 4.447, val loss: 4.483
4.2988786697387695


In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


wdgi3
UIM!5lwvfI91Hgqh
x&_wV!acrSxmeLoARA1!U;XHcO:-texOzMZVAW?s6zZHBv-Xwy6S:c5i1
stbh-h-[Z6!I2ypN_Kc?NN[wE(4741qaPbHM&b[4!'ap
VN)]lGS OqSsCO-4K8NfJ;XEEC6uyDshbumxm:,iik[4[h3r"m(P!_K.8NH0K!aRoY3:c3:tQnB:gBozQRZQ*B72V'D*f
EU;
64[ZLPH
l
O&1B1P[4(?W[ZP'SzV3[j(M GuVw'SN8tek6!J
rS7af7aL3:fX,R(1qWXhyHt705Or
UfX:4[w&G_e &:b
st9]K.n*atOT1bEpl
j&B:&GasP'8'SEpJSaITB:8aLq].(N&j"qR7hp2e]yF4KnCm4(xi]Ds;-PNLLo
4ur
wRC2ff:n?VAB"7UUuku0WZyWMt9]EII']doMlMV8A_m HkJ"qzD!N&ZqgOqW
p[ZfXZsW1[4hb58QBr[nOqnWO4!fUuh; kn[
